# Intro to Interactive Map Layers

**This notebook shows how to perform read & write operations on Interactive Map Layers.**

### Dependencies

* Datasets: Two example datasets will be stored into newly created layers.
* Language: Python

### Test data

Two free datasets (both GeoJSON) are written to interactive map layers.
One contains almost all
[world countries](https://github.com/johan/world.geo.json/blob/master/countries.geo.json)
as Polygon or MultiPolygon geometries, and the other contains the locations
of a number of
[volcanoes](https://earthworks.stanford.edu/download/file/harvard-glb-volc-geojson.json)
all over the world.

### Workflow

- Import packages, incl. HERE Python SDK ones
- Download external datasets
- Create catalog and layers
- Write GeoJSON data into layers
- Show statistical information of layers
- Get single feature from a layer
- Get multiple features from a layer
- Iterate over all the features in a layer
- Search features from a layer based on their properties
- Get features in a bounding box
- Spatial search by providing radius
- Spatial search using Geometry Intersection

## Import Packages

In [ ]:
from datetime import datetime, timezone
import json
import os

import shapely.geometry
from shapely.geometry import Point

from here.platform import Platform
from here.inspector import inspect, new_inspector
from here.inspector.styles import Color

## Download external datasets

In [ ]:
# Download volcanoe data from this source:
# volcanoes of the US [geojson].
# National Atlas of the United States
# https://earthworks.stanford.edu/catalog/stanford-gx245jm9245

# the file is public domain thus available from the sample_datasets folder

In [ ]:
# Download countries data from this source:
# https://github.com/johan/world.geo.json

# the file is public domain thus available from the sample_datasets folder

## Quick-inspect the sample datasets 

In [ ]:
countries_path = "sample_datasets/countries.geo.json"
volcanoes_path = "sample_datasets/stanford-gx245jm9245-geojson.json"

countries = json.load(open(countries_path))
# print(countries)
# print('-------------------------------------------------------------------------')
volcanoes = json.load(open(volcanoes_path))
# print(volcanoes)
# print('-------------------------------------------------------------------------')
insp = new_inspector()
insp.add_features(countries, name="Countries", style=Color.BLUE)
insp.add_features(volcanoes, name="Volcanoes", style=Color.RED)
insp.show()

## Setup

In [ ]:
platform = Platform()

In [ ]:
# create a timestamp/hash-like ID for the catalog ID
ts = datetime.now(timezone.utc).isoformat()[:19]
ts = ts.replace("-", "").replace(":", "").replace("T", "").lower()
print(ts)

## Create resources on the platform

### Create catalog

In [ ]:
cat = platform.create_catalog(id=f"{ts}", name="name", 
                            summary="summary", description="description", billing_tag="ODIN2823")

### Add new Interactive Map layers to catalog

In [ ]:
countries_layer_details = {
    "id": "countries",
    "name": "countries",
    "summary": "Borders of world countries.",
    "description": "Borders of world countries.",
    "layerType": "interactivemap",
    "billingTags": ["ODIN2823"]
}

In [ ]:
volcanoes_layer_details = {
    "id": "volcanoes",
    "name": "volcanoes",
    "summary": "Volcanoes of the world",
    "description": "Volcanoes of the world",
    "layerType": "interactivemap",
    "billingTags": ["ODIN2823"]
}

In [ ]:
platform.update_catalog(
    hrn=cat.hrn, 
    id=f"c{ts}", 
    name="Countries and volcanoes", 
    summary="A catalog for countries and volcanoes.", 
    description="A catalog for countries and volcanoes.", 
    layers=[countries_layer_details, volcanoes_layer_details],
)

In [ ]:
# Global GIS: volcanoes of the world; volcano basic data. [Shapefile].
# American Geological Institute. Retrieved from
# https://earthworks.stanford.edu/catalog/harvard-glb-volc 

volcanoes_layer = cat.get_layer("volcanoes")
volcanoes_layer.write_features(from_file=volcanoes_path)

In [ ]:
countries_layer = cat.get_layer("countries")
countries_layer.write_features(from_file=countries_path)

## Show statistical information of layer

This shows some statitics for for the data in the newly created layers.

In [ ]:
volcanoes_layer.statistics

In [ ]:
countries_layer.statistics

## Get/inspect single feature from layer

In [ ]:
feature = countries_layer.get_feature(feature_id="IND")
inspect(feature, name="Selected feature")

## Get/inspect multiple features from layer

In [ ]:
features = countries_layer.get_features(feature_ids=["IND", "CHN"])
inspect(features, name="Selected features")

## Iterate over & inspect all features in a layer

In [ ]:
features = list(countries_layer.iter_features())
inspect(features, name="All the features")

## Search features from layer based on their properties

In [ ]:
searched_features = countries_layer.search_features(
    params={"p.name": ["India", "United States of America", "Australia", "Germany"]}
)
inspect(searched_features, name="Searched features")

## Get features in a bounding box

In [ ]:
insp = new_inspector()

bbox = (68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078)
bbox_features = countries_layer.get_features_in_bounding_box(bbox)
insp.add_features(bbox_features, name="Features in bounding box")

box_polygon = shapely.geometry.box(*bbox)
insp.add_features(box_polygon, name="Search box", style=Color.GRAY)

insp.show()

## Spatial search within radius

In [ ]:
insp = new_inspector()

radius_search = countries_layer.spatial_search(lat=0, lng=30, radius=1000000)
insp.add_features(radius_search, name="Features within radius")

circle = Point(30, 0).buffer(10)
insp.add_features(circle, name="Search radius", style=Color.GRAY)

insp.show()

## Spatial search using Geometry Intersection

In [ ]:
feature = countries_layer.get_feature(feature_id="USA")
geometry = feature["geometry"]
geometry_search = volcanoes_layer.spatial_search_geometry(geometry=geometry)
inspect(geometry_search, name="Volcanoes in USA", style=Color.RED)

## Clean up

Here, if we like, we can remove all resources generated by this notebook (both locally and on the platform).

In [ ]:
platform.delete_catalog(cat.hrn)

<!--
from urllib.parse import quote, unquote
svg = """<svg width="40" height="48" viewBox="0 0 40 48" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="triangle" d="M11,36.8l-5.5,5.5L0,36.8H11z" fill="#48DAD0"></path><path class="HERE" d="M19.1,24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9,1.7-0.5,2.7,0.5L19.1,24.2z M30.9,9.2c0.9-0.9,1.7-0.5,2.7,0.5
    L31.3,12C30.1,10.7,30.2,9.9,30.9,9.2z M38,11.6c-1.1,1.6-2.9,4.2-4.9,2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4
	c-1.6,1.6-2,3.4-1.5,5.1l-1.6-1.8c-0.5,0.3-2.4,1.9-0.9,4.5l-1.8-1.5l-2.4,2.4l3.2,3.2c-2.5-1.9-5.1-1.8-7.2,0.4
	c-2.3,2.3-2.1,5-0.4,7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9,0.9-1.5,2.2-1.3,3.1L4,24.6l-2.6,2.6l9.6,9.6h5.2l-3.5-3.5
	c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8,1.8-0.3,3.6,1.4l3.4,3.4l2.6-2.6l-3.2-3.2c2.3,1.8,5.1,1.9,7.7-0.6l0,0c1.5-1.4,2-2.8,2-2.8
	l-1.9-1.3c-1.1,1.6-2.9,4.2-4.9,2.3l5-5l3.1,3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5,0-4.1c0.4,0.7,0.9,1.5,1.5,2.1
	c2.5,2.5,5.7,3,8.6,0.2l0,0c1.5-1.4,2-2.8,2-2.8S38,11.6,38,11.6z" fill="#000000"></path></svg>
"""
print(f"""![HERE](data:image/svg+xml,{quote(svg)})
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>""")
-->
![HERE](data:image/svg+xml,%3Csvg%20width%3D%2240%22%20height%3D%2248%22%20viewBox%3D%220%200%2040%2048%22%20fill%3D%22none%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20class%3D%22triangle%22%20d%3D%22M11%2C36.8l-5.5%2C5.5L0%2C36.8H11z%22%20fill%3D%22%2348DAD0%22%3E%3C/path%3E%3Cpath%20class%3D%22HERE%22%20d%3D%22M19.1%2C24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5L19.1%2C24.2z%20M30.9%2C9.2c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5%0A%20%20%20%20L31.3%2C12C30.1%2C10.7%2C30.2%2C9.9%2C30.9%2C9.2z%20M38%2C11.6c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4%0A%09c-1.6%2C1.6-2%2C3.4-1.5%2C5.1l-1.6-1.8c-0.5%2C0.3-2.4%2C1.9-0.9%2C4.5l-1.8-1.5l-2.4%2C2.4l3.2%2C3.2c-2.5-1.9-5.1-1.8-7.2%2C0.4%0A%09c-2.3%2C2.3-2.1%2C5-0.4%2C7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9%2C0.9-1.5%2C2.2-1.3%2C3.1L4%2C24.6l-2.6%2C2.6l9.6%2C9.6h5.2l-3.5-3.5%0A%09c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8%2C1.8-0.3%2C3.6%2C1.4l3.4%2C3.4l2.6-2.6l-3.2-3.2c2.3%2C1.8%2C5.1%2C1.9%2C7.7-0.6l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8%0A%09l-1.9-1.3c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.3l5-5l3.1%2C3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5%2C0-4.1c0.4%2C0.7%2C0.9%2C1.5%2C1.5%2C2.1%0A%09c2.5%2C2.5%2C5.7%2C3%2C8.6%2C0.2l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8S38%2C11.6%2C38%2C11.6z%22%20fill%3D%22%23000000%22%3E%3C/path%3E%3C/svg%3E%0A)
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>